In [1]:
import json
import pandas as pd

In [2]:
# Load JSON file
data = json.load(open('population.json'))

In [3]:
# Interesting data only under GET_STATS_DATA/STATISTICAL_DATA
stat_data = data['GET_STATS_DATA']['STATISTICAL_DATA']

Create two maps for later

In [4]:
column_map = stat_data['CLASS_INF']['CLASS_OBJ']

def eusure_list(x):
    """A helper function - If x is not a list, wrap it with list and return."""
    if isinstance(x, list):
        return x
    else:
        return [x]
    
# Column ID -> Column Name
col_id_to_col_name = {
    x['@id']: x['@name']
    for x in column_map
}

# Column ID -> Column Code -> Column Value
code_to_data = {}
for x in column_map:
    # code_to_data
    x_class = eusure_list(x['CLASS'])
    child_map = {}
    for y in x_class:
        child_map[y['@code']] = y['@name']
        
    code_to_data[x['@id']] = child_map

Load data in JSON (stored as list of dict) into pandas data frame

In [5]:
df = pd.DataFrame(stat_data['DATA_INF']['VALUE'])

Check the first row

In [6]:
df.head()

,$,@area,@cat01,@cat02,@cat03,@tab,@time,@unit
0,126706,00000,000,01000,001,001,1001,千人
1,5320,01000,000,01000,001,001,1001,千人
2,1278,02000,000,01000,001,001,1001,千人
3,1255,03000,000,01000,001,001,1001,千人
4,2323,04000,000,01000,001,001,1001,千人


Now convert values in code (e.g. 01000) into the actual values (e.g. 北海道）

In [7]:
for col_id, mymap in code_to_data.items():
    col_name = col_id_to_col_name[col_id]
    # Create a new column
    df[col_name] = df['@' + col_id].map(mymap)
    # Drop (remove) the original column
    df.drop('@' + col_id, axis=1, inplace=True)

Now the dataframe is more readable

In [8]:
df.head()

,$,@unit,表章項目,男女別,年齢5歳階級,人口,全国・都道府県,時間軸（年月日現在）
0,126706,千人,人口,男女計,総数,総人口,全国,平成29年10月1日現在
1,5320,千人,人口,男女計,総数,総人口,北海道,平成29年10月1日現在
2,1278,千人,人口,男女計,総数,総人口,青森県,平成29年10月1日現在
3,1255,千人,人口,男女計,総数,総人口,岩手県,平成29年10月1日現在
4,2323,千人,人口,男女計,総数,総人口,宮城県,平成29年10月1日現在


Finally, save the dataframe in the CSV format.

`index=False` not to save indices (0, 1, 2... shown at leftmost in the table above.)

In [9]:
df.to_csv('population.csv', index=False)